# Stock market predictions - Decision Trees and Random Forests

Welcome to my rather scuffed implementation of the Decision Tree and Random Forest algorithms for stock market predictions.

The data used in this notebook is sourced from [this Kaggle dataset](https://www.kaggle.com/datasets/borismarjanovic/price-volume-data-for-all-us-stocks-etfs). Data from the GE stock ticker is used in specific, which can be found in the file "ge.us.txt" in CSV format. The dataset includes OHLC prices and the volume, all in 1-day intervals.

## Goals

The project's goal is to use the models above to predict the closing prices of a certain stock ticker. Namely, given the closing prices of